# RISS Crawler
### author: Sung JeeHun
### date: 21.07.26

- code for crawling some paper in RISS

In [ ]:
import urllib.request
import re, time
import requests
import datetime
import pandas as pd
import platform
import pickle

from bs4 import BeautifulSoup
# from openpyxl import load_workbook
from tqdm import tqdm
from pandas import DataFrame
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


import pyautogui as pg


chrome_options = webdriver.ChromeOptions() 
prefs = {"profile.managed_default_content_settings.images": 2} # to unable image
chrome_options.add_experimental_option("prefs", prefs)


if 'Darwin' in platform.system():
    selen_path = '/Users/huni/Dropbox/내 Mac (MacBook-Pro.local)/Downloads/chromedriver'
    chrome_options.add_argument('--kiosk')

elif 'Windows' in platform.system():
    selen_path = r'C:\Users\jhun1\Dropbox\My PC (LAPTOP-VLNR6K8R)\Downloads\chromedriver_win32\chromedriver'
    chrome_options.add_argument('--start-maximized')
# driver = webdriver.Chrome(executable_path = selen_path, chrome_options=chrome_options)


In [ ]:
chrome_options = webdriver.ChromeOptions() 
'''이미지 로드를 안하면, 이후에 초록을 못 긁음'''
# prefs = {"profile.managed_default_content_settings.images": 2} # to unable image
# chrome_options.add_experimental_option("prefs", prefs)
if 'Darwin' in platform.system():
    selen_path = '/Users/huni/Dropbox/내 Mac (MacBook-Pro.local)/Downloads/chromedriver'
    chrome_options.add_argument('--kiosk')

elif 'Windows' in platform.system():
    selen_path = r'C:\Users\jhun1\Dropbox\My PC (LAPTOP-VLNR6K8R)\Downloads\chromedriver_win32\chromedriver'
    chrome_options.add_argument('--start-maximized')

'''
학위논문 제외
학술논문:jn

과학적 소양:ss
과학소양:sk
인문학적 소양:ins
인문 소양: inso
'''
ss_jn = 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EA%B3%BC%ED%95%99%EC%A0%81+%EC%86%8C%EC%96%91&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=%EA%B3%BC%ED%95%99%EC%A0%81+%EC%86%8C%EC%96%91&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=%EA%B3%BC%ED%95%99%EC%A0%81+%EC%86%8C%EC%96%91&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn='
sk_jn = 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EA%B3%BC%ED%95%99%EC%A0%81%EC%86%8C%EC%96%91&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EA%B3%BC%ED%95%99%EC%86%8C%EC%96%91'
ins_jn = 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&query=%EC%9D%B8%EB%AC%B8%ED%95%99%EC%A0%81+%EC%86%8C%EC%96%91&queryText=&iStartCount=0&iGroupView=5&icate=all&colName=re_a_kor&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&pageScale=10&orderBy=&fsearchMethod=search&isFDetailSearch=N&sflag=1&searchQuery=%EC%9D%B8%EB%AC%B8%ED%95%99%EC%A0%81+%EC%86%8C%EC%96%91&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&resultKeyword=%EC%9D%B8%EB%AC%B8%ED%95%99%EC%A0%81+%EC%86%8C%EC%96%91&pageNumber=1&p_year1=&p_year2=&dorg_storage=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&language_code=&ccl_code=&language=&inside_outside=&fric_yn=&image_yn=&regnm=&gubun=&kdc=&ttsUseYn='
inso_jn = 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EA%B3%BC%ED%95%99%EC%86%8C%EC%96%91&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9D%B8%EB%AC%B8%EC%86%8C%EC%96%91'

def riss_crawler(seacrh_url, TITLE, 
                xp_title, xp_year, xp_abstr,
                sel_pt = selen_path, chop = chrome_options):
    driver = webdriver.Chrome(executable_path = sel_pt, chrome_options=chop)

    driver.get(seacrh_url)
    
    '''제약 조건'''
    if TITLE =='과학적 소양':
        temp = driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[4]/div[1]/div[2]/div[3]/div[1]/label').click()
        time.sleep(1)
        but_hund = driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[4]/div[1]/div[2]/div[3]/div[2]/div/ul/li[5]/a').click() # 반응이 오기까지 시간필요
        driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[4]/div[1]/div[2]/button').send_keys(Keys.ENTER)

    else:
        time.sleep(1)
        temp = driver.find_element_by_css_selector('#divContent > div.rightContent > div > div.srchResultW > div:nth-child(1) > div.resultTop2 > div:nth-child(3) > div.selectBoxW > label').click()
        time.sleep(1)
        but_hund = driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[4]/div[1]/div[2]/div[3]/div[2]/div/ul/li[5]/a').click() # 100개씩 출력
        driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[4]/div[1]/div[2]/button').send_keys(Keys.ENTER) # 조회버튼


    '''논문개수 '''
    ppr_count = driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[1]/dl/dd/span/span').text # 검색결과 n개인지 확인
    ppr_count = int(ppr_count)
    print(ppr_count)
    # assert ppr_count == 36, print(ppr_count)

# //*[@id="divContent"]/div[2]/div/div[4]/div[2]/ul/li[2]/div[2]/p[1]/a
    '''제목, 연도, url 크롤링'''
    ss_ds_ls = []
    for j in range(ppr_count):
        url1 = xp_title.replace('here', f'{j % 100 + 1}')
        url2 = xp_year.replace('here', f'{j%100 +1}')
        title = driver.find_element_by_xpath(url1).text
        href = driver.find_element_by_xpath(url1).get_attribute('href')
        url = str(href)
        year = driver.find_element_by_xpath(url2).text
       # try:
        #    if int(year) < 2021-10: continue # pass if published year is longer than 10
        #except:
         #   print('연도추출이 제대로 안됨',title, year, '\n')
          #  break
        ss_ds_ls.append([title, year, url])
        if j % 100 ==0: # 100개 다 긁었을 때 # 새로운 href 형태로 url redirection이 이뤄지는게 아니라, 그냥 자바를 새로 띄우는 형태
            mok = j // 100
            driver.find_element_by_xpath(f'//*[@id="divContent"]/div[2]/div/div[5]/a[{mok+2}]').click() # 7월 크롤링했을 때랑 8월 크롤링했을 때랑 div가 하나씩 밀렸네

    print(ss_ds_ls[:3],'\n', 
        len(ss_ds_ls),'\n\n')


    '''초록 크롤링'''
    for i in range(len(ss_ds_ls)):
        url = ss_ds_ls[i][2]
        driver.get(str(url))
        # 국문 초록이 있는 위치 확인
        addi_pos = 0
        for j in range(1, 4):
            try:
                xp_abstr__ = xp_abstr.replace('[1]', f'[{j}]')
                temp = xp_abstr__[:-2] + '/p'    
                hey = driver.find_element_by_xpath(f'//*[@id="additionalInfoDiv"]/div/div[{j}]/p').text
                if '국문 초록' in hey:
                    addi_pos += j ; del temp # temp를 지워야 뒤에 제대로 크롤링할 때 문제가 없음
                    break

            except NoSuchElementException:
                continue
    
        # 초록 위치를 보고 크롤링하는 것
        if addi_pos == 0: # 없는 경우
            ss_ds_ls[i].append('None')
            continue
        else:
            temp = xp_abstr.replace('[1]', f'[{addi_pos}]')
            try:
                driver.find_element_by_xpath(temp).click() # 더보기 클릭. 시도1
            
            except:
                # from selenium.webdriver import ActionChains 
                # element = driver.find_element_by_xpath(temp) ; driver.execute_script("arguments[0].click();", element) # 더보기 클릭. 시도2
                print(f'초록 안긁힘: {title}')
               #  //*[@id="additionalInfoDiv"]/div/div[1]/a
            # //*[@id="additionalInfoDiv"]/div/div[1]/a
            abstract = driver.find_element_by_xpath(temp[:-2]+'/div[3]/p').text ; del temp


        #print(abstract[:30], '\n')
        ss_ds_ls[i].append(abstract) # 주소 뒤에 추가하기
        if i % 100 == 50:
            print(f'>>>>>> while crawling {TITLE}) {i}, wait..')
            time.sleep(60)
        
    with open(f'./{TITLE}_국내학술논문.pkl', 'wb') as handle:
        pickle.dump(ss_ds_ls, handle, protocol=pickle.HIGHEST_PROTOCOL)

riss_crawler(seacrh_url=ss_jn, TITLE= '과학적 소양',
            xp_title = '//*[@id="divContent"]/div[2]/div/div[4]/div[2]/ul/li[here]/div[2]/p[1]/a',
            xp_year= '//*[@id="divContent"]/div[2]/div/div[4]/div[2]/ul/li[here]/div[2]/p[2]/span[3]',
            xp_abstr= '//*[@id="additionalInfoDiv"]/div/div[1]/a'
                      )


In [ ]:
riss_crawler(seacrh_url=sk_jn, TITLE= '과학소양', 
        xp_title = '//*[@id="divContent"]/div[2]/div/div[4]/div[2]/ul/li[here]/div[2]/p[1]/a',
        xp_year= '//*[@id="divContent"]/div[2]/div/div[4]/div[2]/ul/li[here]/div[2]/p[2]/span[3]',
        xp_abstr= '//*[@id="additionalInfoDiv"]/div/div[1]/a')

In [ ]:
inso_jn = 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EA%B3%BC%ED%95%99%EC%86%8C%EC%96%91&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EC%9D%B8%EB%AC%B8%EC%86%8C%EC%96%91'
riss_crawler(seacrh_url=inso_jn, TITLE= '인문소양', 
            xp_title = '//*[@id="divContent"]/div[2]/div/div[4]/div[2]/ul/li[here]/div[2]/p[1]/a',
            xp_year= '//*[@id="divContent"]/div[2]/div/div[4]/div[2]/ul/li[here]/div[2]/p[2]/span[3]',
            xp_abstr= '//*[@id="additionalInfoDiv"]/div/div[1]/a')